<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/Fujicco_yolo12s_1280_Tray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os
import shutil

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [4]:
# --- 縦横90°切替を学習に混ぜるコールバック ---
import torch, random

def _rotate_xywh_norm(xywh, k):
    x, y, w, h = xywh.unbind(-1)
    if k % 4 == 0:      xn, yn, wn, hn = x,       y,       w, h
    elif k % 4 == 1:    xn, yn, wn, hn = 1.0 - y, x,       h, w   # +90° CCW
    elif k % 4 == 2:    xn, yn, wn, hn = 1.0 - x, 1.0 - y, w, h   # 180°
    else:               xn, yn, wn, hn = y,       1.0 - x, h, w   # 270°
    return torch.stack([xn, yn, wn, hn], dim=-1)

def rotate_batch_90deg_callback(trainer):
    """学習バッチの開始時に、縦横90°/270°入替を確率pで適用"""
    p = 0
    if random.random() > p:
        return
    b = getattr(trainer, "batch", None)
    if b is None:
        return
    imgs = b.get("img", None)
    if imgs is None or imgs.ndim != 4:
        return
    k = random.choice([1, 3])  # 90° or 270°
    b["img"] = torch.rot90(imgs, k=k, dims=[-2, -1])
    if b.get("bboxes", None) is not None and b["bboxes"].numel() > 0:
        b["bboxes"] = _rotate_xywh_norm(b["bboxes"], k)

In [5]:
output_dir = "runs_experiment"  # runs_experimentが既に存在している場合削除
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"✅ ディレクトリ '{output_dir}' を削除しました。")
else:
    print(f"⚠️ ディレクトリ '{output_dir}' は存在しません。")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

⚠️ ディレクトリ 'runs_experiment' は存在しません。
Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [6]:
#!pip install -U torch torchvision torchaudio ultralytics
!pip -q install ultralytics #==8.3.177 --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 975.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

In [7]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

sending incremental file list
dataset.zip
        125.14M   4%   59.66MB/s    0:00:43  rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at rsync.c(716) [generator=3.2.7]

rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at rsync.c(716) [sender=3.2.7]
unzip:  cannot find or open /content/dataset.zip, /content/dataset.zip.zip or /content/dataset.zip.ZIP.
rm: cannot remove '/content/dataset.zip': No such file or directory


In [8]:
# YAML
import yaml

# YAMLファイルを読み込む
data_yaml = "/content/dataset/data.yaml"
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/data.yaml'

In [ ]:
# yolo学習用関数（callbacks= を train() に渡さない）
from ultralytics import YOLO
import os

def train_yolo(
    model_path,
    data_yaml,
    epochs=200,
    batch_size=16,
    img_size=1280,
    output_dir="runs_experiment",
    iou_nms=0.9,
    mosaic=0.15, mixup=0.05, copy_paste=0.6, close_mosaic=15,
    degrees=5.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.2,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.1, fliplr=0.5, flipud=0.0,
    lr0=0.004,
    optimizer="AdamW",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,
    project_name="yolo_project",
    **kwargs,
):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # ← ここでコールバックを登録（train()には渡さない）回転コールバックは使用しない
    #model.add_callback("on_train_batch_start", rotate_batch_90deg_callback)

    os.makedirs(output_dir, exist_ok=True)
    print(f"Training YOLO with {epochs} epochs…")

    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,

        optimizer=optimizer, lr0=lr0, cos_lr=cos_lr, warmup_epochs=warmup_epochs,

        mosaic=mosaic, mixup=mixup, copy_paste=copy_paste, close_mosaic=close_mosaic,
        degrees=degrees, shear=shear, perspective=perspective,
        translate=translate, scale=scale,
        hsv_h=hsv_h, hsv_s=hsv_s, hsv_v=hsv_v,
        erasing=erasing, fliplr=fliplr, flipud=flipud,

        iou=iou_nms,  # ← iou_nms → iou

        amp=amp, workers=workers,
        project=output_dir, name=project_name,

        **kwargs,  # cache, rect, patience などはここで渡す
    )

    print(f"Training complete. Results saved to {output_dir}/{project_name}")


In [ ]:
# ====== 実行パラメータ ======
model_path = "/content/drive/MyDrive/yolo12s.pt"  # or "/content/drive/MyDrive/last.pt"
data_yaml = "/content/dataset/data.yaml"
project_name = "yolo12s_960_learning"
output_dir = "runs_experiment"

train_params = {
    "model_path": model_path,
    "data_yaml": data_yaml,
    "epochs": 300,
    "batch_size": 32,
    "lr0": 0.004,
    "img_size": 960,
    "output_dir": output_dir,
    "cache": "ram",
    "rect": True,
    "iou_nms": 0.60,
    "mosaic": 0.0, "mixup": 0.0, "copy_paste": 0.0, "close_mosaic": 0,
    "degrees": 85.0, "shear": 0.0, "perspective": 0.0, "translate": 0.03, "scale": 0.2,
    "hsv_h": 0.015, "hsv_s": 0.4, "hsv_v": 0.2,
    "erasing": 0.1, "fliplr": 0.2, "flipud": 0.0,
    "optimizer": "AdamW",
    "cos_lr": True,
    "warmup_epochs": 3,
    "amp": True,
    "workers": 2,
    "project_name": project_name,
    # "callbacks": callbacks  ← ここは削除！
}

train_yolo(**train_params)


In [ ]:
#output_dir = "runs_experiment"

best_model_path = f"{output_dir}/{project_name}/weights/best.pt"
last_model_path = f"{output_dir}/{project_name}/weights/last.pt"

from google.colab import files
import os

for model_path in [best_model_path, last_model_path]:
    if os.path.exists(model_path):
        print(f"Downloading: {model_path}")
        files.download(model_path)
    else:
        print(f"[ERROR] {os.path.basename(model_path)} が見つかりません: {model_path}")

